# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_music_db 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project_music_db')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
# this query creates a table for artist, song title and song's length in the music app history that was heard during a particular session and for a specific item_in_session number
query_create_table = "CREATE TABLE IF NOT EXISTS music_library_by_session"

# composite key where session_id is partition key and item_in_session is clustering key
query_create_table = query_create_table + "(session_id INT, item_in_session INT, artist text, song_name TEXT, song_length FLOAT, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query_create_table)
except Exception as e:
    print(e)
               

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query_insert = "INSERT INTO music_library_by_session(session_id, item_in_session, artist, song_name, song_length)"
        query_insert = query_insert + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
# Select the data for session_id=338 and item_in_session=4, to verify that data insertion was correct
quary_select = "SELECT artist, song_name, song_length FROM music_library_by_session WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(quary_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [20]:
# Create a table artist_song_user_sessions for the second query with user_id and session_id as partition and item_in_session as clustering key.
query_create_table = "CREATE TABLE IF NOT EXISTS music_library_by_user_session"

# composite key where (user_id, session_id) is the partition key and item_in_session is the clustering key
query_create_table = query_create_table + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song_name TEXT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query_create_table)
except Exception as e:
    print(e)     

In [23]:
# Insert into music_library_by_user_session for the second query
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query_insert = "INSERT INTO music_library_by_user_session(user_id, session_id, item_in_session, artist, song_name, user_first_name, user_last_name)"
        query_insert = query_insert + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [24]:
# Select name of artist, song and user (first and last name) for userid = 10, sessionid = 182
query_select = "SELECT artist, song_name, user_first_name, user_last_name FROM music_library_by_user_session WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_name, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [25]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_create_table = "CREATE TABLE IF NOT EXISTS user_table_by_song"

# composite key where (song_name, user_id) is the partition key
query_create_table = query_create_table + "(song_name TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, PRIMARY KEY(song_name, user_id))"
try:
    session.execute(query_create_table)
except Exception as e:
    print(e)

                    

In [27]:
# Insert into user_table_by_song for the third query
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query_insert = "INSERT INTO user_table_by_song(song_name, user_id, user_first_name, user_last_name)"
        query_insert = query_insert + "VALUES (%s, %s, %s, %s)"
        session.execute(query_insert, (line[9], line[10] ,line[1], line[4]))

In [28]:
# Select name of firstname, lastname for all users who listened to the song 'All Hands Against His Own'
query_select = "SELECT user_first_name, user_last_name FROM user_table_by_song WHERE song_name='All Hands Against His Own'"
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch


### Drop the tables before closing out the sessions

In [29]:
# Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE music_library_by_session")
    session.execute("DROP TABLE music_library_by_user_session")
    session.execute("DROP TABLE user_table_by_song" )
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()